In [67]:
import pandas as pd
from scipy.special import rel_entr
import numpy as np
import requests
from bs4 import BeautifulSoup
import string
import pyarrow.parquet as pq
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tag import pos_tag

stopwords = set(stopwords.words('english'))

# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('stopwords')

In [17]:
table = pq.read_table('sdsc_data/output.parquet')
df = table.to_pandas()
df.head()

,news,id,collectiondate,title,url,publishdate,author,keywords,src,language,newsindex
0,"Countering Collins’ claims, Democrats said Tru...",1330233,2019-12-18,Partisan rage envelops Congress on the eve of...,http://www.nydailynews.com/news/politics/ny-...,2019-12-18,"[Dave Goldiner, Chris Sommerfeldt]","[rivals, expected, eve, president, volodymyr, ...",http://www.nydailynews.com/,None,None
1,After Cizikas` goal put the Islanders ahead by...,1330234,2019-12-18,Predators score 7 straight goals to beat Isla...,https://www.nydailynews.com/sports/hockey/is...,2019-12-18,[Allan Kreda],"[york, straight, islanders, goal, goals, sweep...",http://www.nydailynews.com/,None,None
2,"It was a tough call over David Fizdale, but Fi...",1330235,2019-12-18,The Daily News’ Knicks all-decade team for th...,http://www.nydailynews.com/sports/basketball...,2019-12-18,[Stefan Bondy],"[mother, daily, knicks, matt, practice, wins, ...",http://www.nydailynews.com/,None,None
3,And it’s a far cry from tactics such as “stamp...,1330236,2019-12-18,Kindergarten student earns enough money from ...,http://www.nydailynews.com/news/national/ny-...,2019-12-18,[Theresa Braine],"[debt, money, students, pay, meal, childs, thr...",http://www.nydailynews.com/,None,None
4,The Hawkeyes had slogged through 17 consecutiv...,1330237,2019-12-18,"Hayden Fry, Texan who turned around Iowa foot...",https://www.nydailynews.com/sports/football/...,2019-12-18,[Ralph D. Russo],"[hawkeyes, texan, unveiled, familiar, worn, un...",http://www.nydailynews.com/,None,None


In [44]:
def find_keywords(text):
    tokens = word_tokenize(period_doc)
    manual_filter = ['’','”','“','—']

    filtered_tokens = [token for token in tokens if token.lower() not in stopwords]
    filtered_tokens = [token for token in filtered_tokens if token not in string.punctuation]
    filtered_tokens = [token for token in filtered_tokens if token not in manual_filter]

    tagged_tokens = pos_tag(filtered_tokens)

    desired_pos = ['NN', 'NNS', 'NNP', 'NNPS']

    keywords = [token for token, pos in tagged_tokens if pos in desired_pos]
    
    return keywords

In [19]:
start_date = pd.to_datetime('2019-12-18')
end_date = pd.to_datetime('2020-12-19')
period = df[(df['collectiondate'] >= start_date) & (df['collectiondate'] <= end_date)][:50]
period_doc = ' '.join(testdf['news'].astype(str))
period_keywords = find_keywords(period_doc)
top_period_keywords = {x[0] : x[1]/len(period_doc) for x in Counter(period_keywords).most_common(5)}

In [60]:
period_keywords = {x[0] : x[1]/len(period_doc) for x in Counter(keywords).most_common(5)}

In [61]:
out = []
for look_date in ['2019-12-18', '2019-12-19']:
    day_df = df[df['collectiondate'] == pd.to_datetime(look_date)][:50]
    day_doc = ' '.join(day_df['news'].astype(str))
    day_keywords = find_keywords(day_doc)
    day_counter = Counter(day_keywords)
    out.append({x : day_counter[x]/len(day_doc) for x in period_keywords})

In [70]:
for dists in out:
    print(sum(rel_entr(list(period_keywords.values()),list(dists.values()))))

0.0
0.0007404775326340417
